In [ ]:
import matplotlib.pyplot as plt
# coding=utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import numpy as np
from keras.layers.core import Flatten
import random,os,sys,shutil
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dropout,LSTM,BatchNormalization
from tensorflow.keras.optimizers import SGD
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
import re
from optimizationAlgorithm import *
from tensorflow.keras.models import load_model

In [ ]:
def FReIndex(colIndex,reIndex):
    dirindex = {}
    lisindex = []
    for i in range(len(colIndex)):
        lisindex.append(i)
    dirindex = dict(zip(lisindex,colIndex))
    colIndex = []
    for i in range(len(reIndex)):
        for key in dirindex:
            if int(key) + 1 == int(reIndex[i]):
                colIndex.append(dirindex[key])
                break
    #print(colIndex)
    return colIndex

In [ ]:
def creat_dataset_X_Y(lis,data_path):
    dataset = pd.read_csv(data_path) 
    dataset = shuffle(dataset)
    Lisclass = dataset['Class'].values
    LisClassNp = []
    for i in range(len(Lisclass)):
        if (Lisclass[i] == 'Kirmizi_Pistachio'):
            LisClassNp.append([0,1])
        else:
            LisClassNp.append([1,0])
    Y = np.array(LisClassNp)
    dataset = dataset.drop(columns='Class')

    colIndex = ['Area','Minor_Axis', 'Eccentricity',
                'Eqdiasq','Convex_Area', 'Aspect_Ratio',
                'Compactness', 'Shapefactor_1', 
                'Shapefactor_3']

    # lis = [] #优化算法参数入口（编码）
    # for i in range(len(colIndex)):
    #     lis.append(len(colIndex)-i)
        
    colIndex = FReIndex(colIndex,lis)
    dataset = pd.DataFrame(dataset,columns=colIndex)
    X = dataset.values
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    return X,Y

In [ ]:
#定义模型
def train_models(X,Y,lis):
    
    sgd = SGD(learning_rate=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    lstm_layers = [3]
    dense_layers = [3]
    units = [32]
    dropout = [0.05]
    Batch_size = [64]
    optimizer = [sgd]
    for the_batch_size in Batch_size:
        for the_dropout in dropout:
            for the_optimizer in optimizer:
                for the_dense_layers in dense_layers:
                    for the_lstm_layers in lstm_layers:
                        for the_units in units:
                            sgd = SGD(learning_rate=0.1, decay=1e-6, momentum=0.9, nesterov=True)
                            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=random.randint(10,100))
                            X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
                            X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
                            model = Sequential()
                            # model.build(input_shape=(277,277,2))
                            #print(model.summary())
                            #model.add(SpatialDropout1D(0.2))
                            model.add(LSTM(the_units ,input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences = True))
                            model.add(Dropout(the_dropout))
                            model.add(BatchNormalization())
                            # #第二层
                            for i in range(the_lstm_layers):
                                model.add(LSTM(the_units,return_sequences=True))
                                model.add(Dropout(the_dropout))
                                model.add(BatchNormalization())

                            model.add(LSTM(the_units))
                            model.add(Dropout(the_dropout))
                            model.add(BatchNormalization())
                            #全连接层
                            for i in range(the_dense_layers):
                                model.add(Dense(the_dense_layers,activation='relu'))
                                model.add(Dropout(the_dropout))
                                
                            # model.add(Flatten()) 
                            
                            
                            model.add(Dense(2, activation='softmax'))
                            
                            #sgd = SGD(learning_rate=0.01, momentum=0.9 , decay=0.1, nesterov=False)
                            
                            # learning_rate = 0.1
                            # decay = 0.001
                            # epochs = 50
                            # batch_size = 64
                            
                            
                            model.compile(  loss='binary_crossentropy',#categorical_crossentropy', binary_crossentropy
                                            optimizer=the_optimizer, metrics=['categorical_accuracy'])
                            #print(model.summary())
                            
                            # if os.path.exists('models'):                         #判断是否存在输入的文件夹，存在则继续
                            #     if os.path.exists('models/' + str(lis)):    
                            #         pass#判断否存在输入的子文件夹
                            #         #print('两个文件都已经存在')
                            #     else:                                                                   # 文件夹存在，子文件夹不存在，则创建
                            #         os.mkdir('models/' + str(lis))                                              #创建子文件夹
                            #         #print('{}文件夹创建成功'.format('models/' + str(lis)))                       #打印XX文件夹创建成功
                            # else:                                                                #文件夹不存在，则创建文件夹和子文件夹
                            #     os.mkdir('models')                                                #创建文件夹
                            #     os.mkdir('models/' + str(lis))                                                  #创建子文件夹
                            #     #print('{}文件夹和{}文件夹创建成功'.format(desk_file_name,zi_file_name))
                                
                                
                            epochs = 100
                            batch_size = the_batch_size
                            if(the_optimizer == sgd):
                                the_optimizer = 'sgd'
                            filename = str(lis)
                            filepath = 'models/' + filename + '/{categorical_accuracy:.4f}.h5'
                            checkpoint = ModelCheckpoint(
                                                filepath=filepath,
                                                save_weights_only=False,
                                                monitor='categorical_accuracy',
                                                mode='max',
                                                save_best_only=True)
                            history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1,
                                                #callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])
                                                verbose=0,callbacks=checkpoint)

In [ ]:
def max_acc(path):
    for root, dirs, files in os.walk(path):
        pass
    name = files[-1]
    return name

In [ ]:
#print(type(max_acc('models')))
# './Pistachio_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.csv'

In [ ]:
def acc_source(lis,filepath):
    
    best_model = load_model(filepath)
    X,Y = creat_dataset_X_Y(lis,'test.csv')

    #进行预测 make a prediction
    X = X.reshape(X.shape[0],1,X.shape[1])
    yhat = best_model.predict(X)

    # print(yhat)
    # print(Y)
    sorce = 0
    for i in range(len(Y)):
        sorce += roc_auc_score(Y[i], yhat[i])

    print('ACC:',sorce/len(Y))
    return -sorce/len(Y)


In [ ]:
def enSimpleCode(lis):
    X,Y = creat_dataset_X_Y(lis,'./Pistachio_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.csv')
    train_models(X,Y,lis)
    path = 'models/' + str(lis)  + '/' + max_acc('models/' + str(lis))
    return round(acc_source(lis,path),4) * 10000

In [ ]:
shutil.rmtree('models')  
os.mkdir('models')
ColIndex = ['Area','Minor_Axis', 'Eccentricity',
                'Eqdiasq','Convex_Area', 'Aspect_Ratio',
                'Compactness', 'Shapefactor_1', 
                'Shapefactor_3']
optAlgo = OptimizationAlgorithm(ColIndex)
optAlgo.ga(enSimpleCode, len(ColIndex), 100, 2)

#[9, 5, 1, 2, 6, 3, 4, 7, 8]
